In [1]:
import numpy as np

In [2]:
n = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]

In [3]:
# p - persistance
# r - ready time
# d - oczekiwany termin zakończenia
# w - weight
# Cj oznacza moment zakończenia wykonywania zadania Jj w uszeregowaniu

# p r d w
# r  >= c - p

# Generate

In [50]:
for x in n:
    time = 0
    fileData = []
    fileData.append(x)
    for _ in range(x):
        p = np.random.randint(1, 10)
        r = np.random.randint(time - 5 if time - 5 > 0 else 0 ,time + 5)
        d = np.random.randint(r+p, r+p+10)
        w = np.random.randint(0,10)
        time += p
        
        fileData.append([p,r,d,w])
    with open(f'data/instances_{x}.csv', "w") as f:
        f.write(f"{str(fileData[0])}\n")
        for line in fileData[1:]:            
            f.write(f"{' '.join([str(x) for x in line])}\n")

# Verify

In [64]:
x = 50

In [84]:
data = {}

with open(f'data/result_{x}.csv', "r") as result:
    res = result.read().split('\n')
    weight = int(res[0])
    taskQueue = [int(t) for t in res[1:-1]]
    
    with open(f'data/instances_{x}.csv', "r") as instance:
        d = instance.read().split('\n')
        n = int(d[0])
        for i,line in enumerate(d[1:]):
            if line == '':
                continue
            p,r,d,w = line.split(' ')
            p,r,d,w = int(p), int(r), int(d), int(w)
            data[i] = {"p": p, "r": r, "d": d, "w": w, "c": 0 }
    
        

In [85]:
calculatedWeight = 0
time = 0
for task in taskQueue:
    if time < data[task]['r']:
        time = data[task]['r']
    # add p time to current time
    time += data[task]['p']
    c = time
    data[task]['c'] = c
    
    # check correctness
    if not data[task]["r"] <= data[task]["c"] - data[task]["p"]: 
        print("Error")
    
    # verify cost
    if data[task]["c"] > data[task]["d"]:
        calculatedWeight += data[task]["w"]
if calculatedWeight != weight:
    print(f"wrong weight. Calculated: {calculatedWeight}, read: {weight}")
else:
    print(f"Weight: {calculatedWeight} confirmed")

Weight: 93 confirmed
